# Python Anti patterns 

## correctness


### No exception type(s) specified
The function divide simply divides a by b. To avoid invalid calculations (e.g., a division by zero), a try-except block is added. This is valid and ensures that the function always returns a result. However, by securing your code with the try clause, you might hide actual programming errors, e.g., that you pass a string or an object as b, instead of a number. By not specifying an exception type, you not only hide this error but you also lose information about the error itself.

#### Anti-pattern

In [ ]:
def divide(a, b):
    try:
        result = a / b
    except:
        result = None
    return result


In [ ]:
result = divide(92,0)
result

#### Best practice

In [ ]:
def divide(a, b):
    result = None
    try:
        result = a / b
    except ZeroDivisionError:
        print("Type error: division by 0.")
    except TypeError:
        # E.g., if b is a None
        print("Type error: division by '{0}'.".format(b))
    except Exception as e:
        # handle any other exception
        print("Error '{0}' occured. Arguments {1}.".format(e.message, e.args))
    else:
        # Excecutes if no exception occured
        print("No errors")
    finally:
        # Executes always
        if result is None:
            result = 0
    return result

In [ ]:
result = divide(92,2)
result

In [ ]:
from typing import Optional
from numbers import Number

def divide(a: Number, b: Number) -> Optional[float]:
    """Divide two numbers safely handling different types of exceptions.

    This function performs division between two numbers and handles various exceptions
    that might occur during the operation.

    Args:
        a (Number): The dividend (numerator) for the division
        b (Number): The divisor (denominator) for the division

    Returns:
        Optional[float]: The result of the division a/b. Returns 0 if any error occurs.

    Raises:
        ZeroDivisionError: If b is 0
        TypeError: If any of the inputs is not a number or is None
        Exception: For any other unexpected errors

    Examples:
        >>> divide(10, 2)
        5.0
        >>> divide(10, 0)  # Returns 0, prints error message
        0
        >>> divide(10, None)  # Returns 0, prints error message
        0
    """
    result: Optional[float] = None
    try:
        result = float(a / b)
    except ZeroDivisionError:
        print("Type error: division by 0.")
    except TypeError:
        # E.g., if b is a None
        print("Type error: division by '{0}'.".format(b))
    except Exception as e:
        # handle any other exception
        print("Error '{0}' occurred. Arguments {1}.".format(str(e), e.args))
    else:
        # Executes if no exception occurred
        print("No errors")
    finally:
        # Executes always
        if result is None:
            result = 0
    return result

In [ ]:

result = divide(92,0)
result

## Maintainability

### Not using with to open files
In Python 2.5, the file class was equipped with special methods that are automatically called whenever a file is opened via a with statement (e.g. with open("file.txt", "r") as file). These special methods ensure that the file is properly and safely opened and closed.

#### Anti-pattern

In [ ]:
f = open("file.txt", "r")
content = f.read()
1 / 0 # ZeroDivisionError
# never executes, possible memory issues or file corruption
f.close()

#### Best practice

In [ ]:
with open("file.txt", "r") as f:
    content = f.read()
    # Python still executes f.close() even though an exception occurs
    1 / 0

In [ ]:
from typing import Any, Dict, Union
from pathlib import Path
import json
import logging
from json.decoder import JSONDecodeError

def read_json_file(file_path: Union[str, Path]) -> Dict[str, Any]:
    """Read and parse a JSON file, handling potential errors gracefully.

    This function attempts to read and parse a JSON file from the given path.
    It handles various potential errors including file not found, permission issues,
    and invalid JSON format. In case of any error, it returns an empty dictionary
    and logs the error.

    Args:
        file_path (Union[str, Path]): Path to the JSON file. Can be either a string
            or a Path object.

    Returns:
        Dict[str, Any]: The parsed JSON content as a dictionary. Returns an empty
            dictionary in case of any error.

    Raises:
        No exceptions are raised as they are caught and logged internally.

    Examples:
        >>> data = read_json_file("config.json")
        >>> if data:  # Check if data was loaded successfully
        ...     print(data["setting"])
        >>> # Handle empty case
        >>> data = read_json_file("nonexistent.json")  # Returns {}

    Note:
        The function will log errors using the logging module instead of raising
        exceptions. Make sure to configure logging if you want to capture these
        messages.
    """

    file_path = Path(file_path)
    result: Dict[str, Any] = {}
    
    try:
        with Path(file_path).open('r', encoding='utf-8') as file:
            result = json.load(file)
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
    except PermissionError:
        logging.error(f"Permission denied accessing file: {file_path}")
    except JSONDecodeError as e:
        logging.error(f"Invalid JSON format in {file_path}: {str(e)}")
    except UnicodeDecodeError:
        logging.error(f"File encoding error in {file_path}. Expected UTF-8")
    except IsADirectoryError:
        logging.error(f"Path is a directory, not a file: {file_path}")
    except Exception as e:
        logging.error(f"Unexpected error reading {file_path}: {str(e)}")
    
    return result

## Readability

### Not using named tuples when returning more than one value from a function
Named tuples can be used anywhere where normal tuples are acceptable, but their values can be accessed through their names in addition to their indexes. This makes the code more verbose and readable.

#### Anti-pattern

The code below returns a first name, middle name, and last name using a normal, unnamed tuple. After calling the tuple, each value can only be returned via an index. 
This code is difficult to use: the caller of the function has to know that the first element is the first name, the second is the middle name, and the third is the last name.

In [ ]:
def get_name():
    return "Richard", "Xavier", "Jones"

name = get_name()

# no idea what these indexes map to!
print(name[0], name[1], name[2])

#### Best practice

In [ ]:
from collections import namedtuple

def get_name():
    name = namedtuple("name", ["first", "middle", "last"])
    return name("Richard", "Xavier", "Jones")

name = get_name()

# much easier to read
print(name.first, name.middle, name.last)

In [ ]:
from typing import NamedTuple, Union, Tuple

class RGB(NamedTuple):
    """Named tuple representing RGB color values.
    
    Attributes:
        red (int): Red component (0-255)
        green (int): Green component (0-255)
        blue (int): Blue component (0-255)
    """
    red: int
    green: int
    blue: int

def parse_color(
    color: Union[str, Tuple[int, int, int], str]
) -> RGB:
    """Convert different color formats to RGB named tuple.

    Converts hex colors or RGB tuples to a standardized RGB named tuple.
    Ensures all values are within valid range (0-255).

    Args:
        color: Input color in any of these formats:
            - Hex string: "#FF0000" or "FF0000"
            - RGB tuple: (255, 0, 0)
            - Color name: "red", "blue", "green"

    Returns:
        RGB: Named tuple with red, green, and blue values

    Examples:
        >>> color = parse_color("#FF0000")
        >>> print(f"Red: {color.red}, Green: {color.green}, Blue: {color.blue}")
        Red: 255, Green: 0, Blue: 0
        
        >>> rgb = parse_color((100, 150, 200))
        >>> print(rgb)
        RGB(red=100, green=150, blue=200)
    """
    # Basic color dictionary
    COLOR_NAMES = {
        'red': (255, 0, 0),
        'green': (0, 255, 0),
        'blue': (0, 0, 255),
        'white': (255, 255, 255),
        'black': (0, 0, 0),
    }
    
    try:
        # Handle hex strings
        if isinstance(color, str):
            # Remove '#' if present
            color = color.lstrip('#').lower()
            
            # Handle color names
            if color in COLOR_NAMES:
                return RGB(*COLOR_NAMES[color])
            
            # Convert hex to RGB
            if len(color) == 6:
                r = int(color[0:2], 16)
                g = int(color[2:4], 16)
                b = int(color[4:6], 16)
                return RGB(r, g, b)
        
        # Handle RGB tuples
        elif isinstance(color, tuple) and len(color) == 3:
            r, g, b = [max(0, min(255, c)) for c in color]  # Clamp values
            return RGB(r, g, b)
            
        raise ValueError("Invalid color format")
        
    except Exception as e:
        # Return black on error
        return RGB(0, 0, 0)

In [ ]:
# Using hex color
red = parse_color("#FF0000")
print(f"Red color: {red}")  # RGB(red=255, green=0, blue=0)

# Using RGB tuple
custom = parse_color((100, 150, 200))
print(f"Custom color: {custom}")  # RGB(red=100, green=150, blue=200)

# Using color name
blue = parse_color("blue")
print(f"Blue values - R: {blue.red}, G: {blue.green}, B: {blue.blue}")

# Access individual components
print(f"Red component: {custom.red}")
print(f"Green component: {custom.green}")
print(f"Blue component: {custom.blue}")

# Error handling
invalid = parse_color("not a color")
print(f"Invalid color returns black: {invalid}")  # Returns RGB(0, 0, 0)

## Performance & security

### Using key in list to check if key is contained in list
Using key in list to iterate through a list can potentially take n iterations to complete, where n is the number of items in the list. If possible, you should change the list to a set or dictionary instead, because Python can search for items in a set or dictionary by attempting to directly accessing them without iterations, which is much more efficient.

#### Anti-pattern

In [ ]:
l = [1, 2, 3, 4]

# iterates over three elements in the list
if 0 in l:
    print("The number 3 is in the list.")
else:
    print("The number 3 is NOT in the list.")

#### Best practice

**Use a set or dictionary instead of a list**

In [ ]:
s = set([1, 2, 3, 4])

if 3 in s:
    print("The number 3 is in the list.")
else:
    print("The number 3 is NOT in the list.")

### use of exec

In [ ]:
s = "print(\"Hello, World!\")"
exec(s)

In [ ]:
def print_hello_world():
    print("Hello, World!")

print_hello_world()

```python
for key, val in values:
    fieldName = valueToFieldName[key]
    fieldType = fieldNameToType[fieldName]
    if fieldType is int:
        s = 'object.%s = int(%s)' % (fieldName, fieldType) 
    #Many clauses like this...

exec(s)
```